In [1]:
from datetime import datetime, date, timedelta
from twilio.rest import Client
import time
import pytz

import os
from dotenv import load_dotenv
load_dotenv('./.env')

from sqlalchemy import create_engine, exc, update, MetaData, Table, Column, Integer
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy.dialects.postgresql import UUID

import pandas as pd
import numpy as np


from twilio.rest import Client
from twilio.base.exceptions import TwilioRestException
import re



In [2]:
# PostgreSQL connection details
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_PORT = os.getenv("DB_PORT")

In [3]:
# Twilio account details
TWILIO_ACCOUNT_SID = os.getenv("TWILIO_ACCOUNT_SID")
TWILIO_AUTH_TOKEN = os.getenv("TWILIO_AUTH_TOKEN")
TWILIO_PHONE_NUMBER = os.getenv("TWILIO_PHONE_NUMBER")
TWILIO_MESSAGING_SERVICE_SID = os.getenv("TWILIO_MESSAGING_SERVICE_SID")

In [9]:
# Create the SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

In [5]:
today = date.today()
tomorrow = today + timedelta(days=1)
# Format the dates as strings
today_str = today.strftime('%Y-%m-%d')
# print(today_str)
tomorrow_str = tomorrow.strftime('%Y-%m-%d')

In [12]:
query = f'''
            SELECT "Id" AS "Id_Ev", "UserId", "DateStartEvent", "HoursStartEvent", "DateEndEvent", "HoursEndEvent", "Notify", "StatoEvento", "EventId", "notifysend", "messagesid"
            FROM "public"."Calendar"
            WHERE "DateStartEvent" IN ('{today_str}', '{tomorrow_str}')
        '''
df_calendar_all = pd.read_sql_query(query, engine)


In [13]:
df_calendar_all

,Id_Ev,UserId,DateStartEvent,HoursStartEvent,DateEndEvent,HoursEndEvent,Notify,StatoEvento,EventId,notifysend,messagesid
0,110,d5afa28c-8bf6-4ca1-9059-72298ad83e87,2023-07-17,15:11:52,2023-07-17,16:11:52,0 days 00:05:00,Created,9bc8ef16-833b-4730-a98f-a1058862d142,1,SMb6057f036446e652481c282b4118483f
1,193,cc19af0f-dbf8-4d00-ba84-1860b0e33e0e,2023-07-18,14:29:54,2023-07-18,15:29:54,0 days 00:05:00,Created,17cae7c1-5817-4f46-aba6-5f4d95d1af05,0,None
2,162,d5afa28c-8bf6-4ca1-9059-72298ad83e87,2023-07-18,15:17:00,2023-07-18,16:17:00,0 days 00:05:00,Created,8e12abcd-e95f-41d2-be70-f7382be41ea3,0,None
3,108,d5afa28c-8bf6-4ca1-9059-72298ad83e87,2023-07-18,15:11:52,2023-07-18,16:11:52,0 days 00:05:00,Created,9f12e21f-fff0-4b5b-a69a-de5860943271,0,None
4,230,e9094473-8ac3-4a8b-9027-2671717783c5,2023-07-17,11:00:21,2023-07-17,12:00:21,0 days 00:30:00,Cancelled,8f7c1b8b-16a7-47ec-96f3-a5eb495d4795,0,None
5,231,e9094473-8ac3-4a8b-9027-2671717783c5,2023-07-17,11:00:21,2023-07-17,12:00:21,0 days 00:30:00,Cancelled,f793887b-e592-4ff4-8647-5d5ce195e618,0,None
6,235,e9094473-8ac3-4a8b-9027-2671717783c5,2023-07-17,19:22:04,2023-07-17,20:45:04,0 days 00:30:00,Created,e2d15db5-e762-4cbb-9fb2-889618e0af9a,1,SMbecec956f4c11c6ba363eb3bd3b54ce7
7,221,e9094473-8ac3-4a8b-9027-2671717783c5,2023-07-18,10:00:34,2023-07-18,11:00:34,0 days 00:30:00,Cancelled,31b77354-8b1f-475a-aa0b-3735b772fd51,0,None
8,220,e9094473-8ac3-4a8b-9027-2671717783c5,2023-07-17,10:00:34,2023-07-17,11:00:34,0 days 00:30:00,Cancelled,03a1d212-e3bf-46bc-803e-2cc3d6218fe2,0,None
9,224,e9094473-8ac3-4a8b-9027-2671717783c5,2023-07-17,11:00:21,2023-07-17,12:00:21,0 days 00:30:00,Cancelled,2eb784d8-9f53-4ed0-a1ef-7d1956c2bc1d,0,None


In [7]:
# Function to check and add country code
def add_country_code(phone_number):
    if re.match(r"^\+\d{1,}", phone_number):
        return phone_number  # Country code already present
    else:
        if len(phone_number) == 10:
            return "+39" + phone_number
        elif len(phone_number) > 10:
            return "+" + phone_number  # Add country code

# Check the state of the engine by attempting a simple query
def check_engine_state():
    try:
        # Attempt a simple query to check the connection
        with engine.connect():
            pass
        return True  # Connection is active
    except (exc.DBAPIError, exc.OperationalError):
        return False  # Connection is closed or in a disconnected state

# Schedule a message based on the provided row
def schedule_message(row):
    to_number = row["MobilePhone"]
    sending_time = row["NotifyDateTimeUTC"]
    notification_enabled = row["Notification"]
    body = f"Your scheduled event at AVTI: https://avti.app/beta/navigation?id={row['UserId']}&eventId={row['EventId']}" 


    if notification_enabled:
        try:
            if validate_sending_time(sending_time):
                # print("validate_sending_time", validate_sending_time(sending_time))
                message = client.messages \
                    .create(
                        messaging_service_sid = TWILIO_MESSAGING_SERVICE_SID,
                        to = to_number,
                        body = body,
                        schedule_type = 'fixed',
                        send_at = sending_time
                    )
                print(f"The message for the EventId:{row['EventId']} has been scheduled with the following message sid: {message.sid}")
                df_calendar.at[row.name, "messagesid"] = message.sid
                df_calendar.at[row.name, "notifysend"] = 1
                # print(row['EventId'])
                # print(sending_time)
            else: pass
        except TwilioRestException as e:
            print(f"An error occurred while sending the message to following UUID({row['UUID']}) with following EventId({row['EventId']}): {str(e)}")
    else:
        print(f"Skipping notification for {to_number} as 'Notification' is False.")
    print()

def update_event_status(row):
    # Update message status
    message_sid = row['messagesid']
    
    try:
        message = client.messages(message_sid).update(status='canceled')

        # Update "notifysend" in DataFrame
        index = row.name
        df_calendar_notifysend_1.at[row.name, "notifysend"] = 0
        print("Cancelled message sid: ",message_sid)
    except TwilioRestException as e:
        print(f"An error occurred while cancelling the message to following UUID({row['UUID']}) with following EventId({row['EventId']}): {str(e)}")

    
    
# Validate the sending time and return the sending time if it meets the conditions
def validate_sending_time(sending_time):
    current_time = datetime.utcnow()
    # sending_time = datetime(2023, 6, 30, 11, 35, 10)  # Replace with your sending_time

    time_difference = sending_time - current_time
    # print(time_difference)

    if time_difference > timedelta(minutes=15) and time_difference < timedelta(days=7):
        return sending_time
    else:
        return False

def convert_to_iso8601(datetime_string):
    # Create a datetime object from the string
    # italian_datetime = datetime.strptime(datetime_string, "%Y-%m-%d %H:%M:%S")

    # Define the timezone for Italy (Europe/Rome)
    italian_timezone = pytz.timezone('Europe/Rome')

    # Localize the Italian datetime to the Italian timezone
    localized_datetime = italian_timezone.localize(datetime_string)

    # Convert the localized datetime to UTC
    utc_datetime = localized_datetime.astimezone(pytz.utc)

    # Format the UTC datetime in ISO 8601 format
    iso_8601_format = utc_datetime.strftime('%Y-%m-%d %H:%M:%S')

    return iso_8601_format

Base = declarative_base()
class Calendar(Base):
    __tablename__ = 'Calendar'
    __table_args__ = {'schema': 'public'}
    
    Id = Column(Integer, primary_key=True)
    notifysend = Column(Integer)
    EventId = Column(UUID(as_uuid=True))
    messagesid = Column(Integer)

### Add these lines to the final code!!

In [17]:
while True:    
    max_attempts = 3  # Maximum number of attempts
    num_attempts = 0  # Counter variable
    while num_attempts < max_attempts:
        try:
            if not check_engine_state():
                # Reopen the engine if the connection is closed or disconnected
                engine.dispose()
                engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
                
            # "df_calendar_all" is the table extracted from the database, which after will be devided into "df_calendar_notifysend_1" and "df_calendar" tables
            ##################################################################################################################################################
            # Get today's and tomorrow's dates
            today = date.today()
            tomorrow = today + timedelta(days=1)

            # Format the dates as strings
            today_str = today.strftime('%Y-%m-%d')
            tomorrow_str = tomorrow.strftime('%Y-%m-%d')

            # Create a DataFrame from the Calendar table with filters
            query = f'''
                SELECT "UserId", "DateStartEvent", "HoursStartEvent", "DateEndEvent", "HoursEndEvent", "Notify", "StatoEvento", "EventId", "notifysend", "messagesid"
                FROM "public"."Calendar"
                WHERE "DateStartEvent" IN ('{today}', '{tomorrow}')
            '''
            df_calendar_all = pd.read_sql_query(query, engine)
            ##################################################################################################################################################


            # "df_calendar_notifysend_1" this table should be used in order to figure out if the previously scheduled events got calncelled.
            ##################################################################################################################################################
            df_calendar_notifysend_1 = df_calendar_all[(df_calendar_all['notifysend'] == 1) & (df_calendar_all['StatoEvento'] == "Cancelled")]
            # Apply the function to each row of the DataFrame
            df_calendar_notifysend_1.apply(update_event_status, axis=1) 
            # Create a session
            # Replace 'your_connection_string' with your PostgreSQL connection string
            Session = sessionmaker(bind=engine)
            session = Session()
            # Base = declarative_base()

            try:
                # Update the rows in the "public"."Calendar" table
                for index, row in df_calendar_notifysend_1.iterrows():
                    event_id = row["EventId"]
                    notifysend_value = row["notifysend"]

                    # Define the update query
                    stmt = update(Calendar).where(Calendar.EventId == event_id).values(notifysend =notifysend_value)
                    session.execute(stmt)
                    session.commit()
            
            except Exception as e:
                print("An error occurred:", str(e))

            session.close()
            ##################################################################################################################################################


            # "df_calendar" this table should be used to schedule the  new events
            ##################################################################################################################################################
            df_calendar = df_calendar_all[(df_calendar_all['notifysend'] == 0) & (df_calendar_all['StatoEvento'] == "Created")]
            df_calendar = df_calendar.rename(columns={'UserId': 'UUID'})

            # Create a DataFrame from the AspNetUsers table
            df_aspnetusers = pd.read_sql_query('SELECT "Id", "UserId" FROM "public"."AspNetUsers"', engine)
            df_aspnetusers = df_aspnetusers.rename(columns={"Id":"UUID"})

            # Perform left join
            df_calendar = pd.merge(df_calendar, df_aspnetusers, on='UUID', how='left')

            # Create a DataFrame from the Users table
            # df_users = pd.read_sql_query('SELECT "Id","Surname","Name", "MobilePhone", "Email", "StatusUser","Language", "Notify" FROM "public"."Users"', engine)
            df_users = pd.read_sql_query('SELECT "Id", "MobilePhone", "StatusUser","Language", "Notify" FROM "public"."Users"', engine)
            df_users = df_users.rename(columns={"Id":"UserId", "Notify":"Notification"})

            # Perform left join
            df_calendar = pd.merge(df_calendar, df_users, on='UserId', how='left')

            df_calendar = df_calendar[(df_calendar['StatusUser'] == "Active") & (df_calendar['Notification'] == True)]

            df_calendar["DateStartEvent"] = pd.to_datetime(df_calendar["DateStartEvent"])  # Convert to datetime if not already done

            # Convert 'HoursStartEvent' to string representation
            df_calendar["HoursStartEvent"] = df_calendar["HoursStartEvent"].apply(lambda x: x.strftime('%H:%M:%S'))

            # Concatenate 'DateStartEvent' and 'HoursStartEvent'
            df_calendar["DateTimeStartEvent"] = pd.to_datetime(df_calendar["DateStartEvent"].astype(str) + ' ' + df_calendar["HoursStartEvent"])

            # new column 'NotifyDateTime' containing the adjusted datetime values
            df_calendar["NotifyDateTime"] = df_calendar["DateTimeStartEvent"] - df_calendar["Notify"]

            display(df_calendar)
            # Get today's date
            # Filter the DataFrame based on the date part of 'NotifyDateTime'
            df_calendar = df_calendar[df_calendar['NotifyDateTime'].dt.date == today]
            df_calendar = df_calendar.reset_index(drop=True)
            df_calendar["NotifyDateTimeUTC"] = df_calendar["NotifyDateTime"].apply(convert_to_iso8601)
            df_calendar["NotifyDateTimeUTC"] = pd.to_datetime(df_calendar["NotifyDateTimeUTC"])

            # Apply the function to the "MobilePhone" column
            df_calendar["MobilePhone"] = df_calendar["MobilePhone"].apply(add_country_code)
            # Replace None values with NaN in the "MobilePhone" column
            df_calendar["MobilePhone"].fillna(np.nan, inplace=True)
            df_calendar = df_calendar.dropna(subset=["MobilePhone"])

            # Schedule the Event in the Twilio
            df_calendar.apply(schedule_message, axis=1)

            # Updating the Postgresql Calendar table
            filtered_df = df_calendar[df_calendar["notifysend"] == 1]
            

            # Replace 'your_connection_string' with your PostgreSQL connection string
            Session = sessionmaker(bind=engine)
            session = Session()

            # Base = declarative_base()
            
            try:
                # Update the rows in the "public"."Calendar" table
                for index, row in filtered_df.iterrows():
                    event_id = row["EventId"]
                    notifysend_value = row["notifysend"]
                    new_messagesid = row["messagesid"]

                    stmt = update(Calendar).where(Calendar.EventId == event_id).values(messagesid=new_messagesid, notifysend =notifysend_value)
                    session.execute(stmt)
                    session.commit()

            except Exception as e:
                print("An error occurred:", str(e))
            
            session.close()

            break
        
        except exc.OperationalError as e:
            num_attempts += 1
            print(f"Attempt {num_attempts} failed. Error: {str(e)}")
            if num_attempts == max_attempts:
                # If the maximum number of attempts is reached, raise the exception
                raise
            else:
                # Wait for some time before retrying
                # You can adjust the sleep time according to your needs
                time.sleep(1)  # Import the time module for this approach
    
    # Sleep for 5 minutes
    time.sleep(60)

,UUID,DateStartEvent,HoursStartEvent,DateEndEvent,HoursEndEvent,Notify,StatoEvento,EventId,notifysend,messagesid,UserId,MobilePhone,StatusUser,Language,Notification,DateTimeStartEvent,NotifyDateTime
0,5ebdb461-4d27-49ca-a6a1-4b2c0b00c1a5,2023-07-08,14:48:14,2023-07-08,15:48:14,0 days 00:30:00,Created,827bc531-3c44-4ef3-bc61-50eb150403b5,0,None,74,3924162864,Active,it,True,2023-07-08 14:48:14,2023-07-08 14:18:14
2,8aef6584-c948-48a7-ba92-ce72b09119c1,2023-07-07,13:47:42,2023-07-07,14:47:42,0 days 01:00:00,Created,62028a31-b48a-4594-a19b-c72a915f6249,0,None,1,12345,Active,it,True,2023-07-07 13:47:42,2023-07-07 12:47:42


KeyboardInterrupt: 

In [14]:
l = ["SM3d3bf7bc76e344ef535e22d0eb41e6ef"]
for i in l:
    message = client.messages(i) \
                    .update(status='canceled')

# msg_ctx = client.messages.get("SM1f2a8ebf4d746b465019050fd20ee32d").fetch()
# vars(msg_ctx)